In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, \
    Qwen2_VL_process_fn, LLAVA_QWEN2, FastVLM_process_fn

from PIL import Image
import numpy as np
import torch
import math
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

from transformers.image_transforms import (
    convert_to_rgb,
    resize,
)

/workspace/VLM_Embed/vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/workspace/VLM_Embed/vlm/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM_Embed/src/model/vlm_backbone/internvideo2/modeling_internvideo2.py:541: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


FusedMLP of flash_attn is not installed!!!
DropoutAddRMSNorm of flash_attn is not installed!!!
flash_attn_interface or bert_padding of flash_attn is not installed!!!


[2026-01-01 10:34:20,262] DEBUG [matplotlib:342] matplotlib data path: /workspace/VLM_Embed/vlm/lib/python3.11/site-packages/matplotlib/mpl-data
[2026-01-01 10:34:20,271] DEBUG [matplotlib:342] CONFIGDIR=/root/.config/matplotlib
[2026-01-01 10:34:20,284] DEBUG [matplotlib:1560] interactive is False
[2026-01-01 10:34:20,286] DEBUG [matplotlib:1561] platform is linux
[2026-01-01 10:34:20,317] DEBUG [matplotlib:342] CACHEDIR=/root/.cache/matplotlib
[2026-01-01 10:34:20,321] DEBUG [matplotlib.font_manager:1635] Using fontManager instance from /root/.cache/matplotlib/fontlist-v390.json
[2026-01-01 10:34:20,707] DEBUG [matplotlib.pyplot:496] Loaded backend module://matplotlib_inline.backend_inline version unknown.
[2026-01-01 10:34:20,712] DEBUG [matplotlib.pyplot:496] Loaded backend inline version unknown.


In [2]:
model_args = ModelArguments(
    model_name='Qwen/Qwen2-VL-2B',
    checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-2B',
    pooling='last',
    normalize=True,
    model_backbone='qwen2_vl',
    lora=True
)
data_args = DataArguments()

processor = load_processor(model_args, None)
model = MMEBModel.load(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

[2026-01-01 10:34:20,736] INFO [src.utils:21] Loading processor from: TIGER-Lab/VLM2Vec-Qwen2VL-2B
[2026-01-01 10:34:20,758] DEBUG [urllib3.connectionpool:1049] Starting new HTTPS connection (1): huggingface.co:443


Load Qwen2-VL processor
>>>>>>>>>>>>>>>>>>>>>>>> Processor TIGER-Lab/VLM2Vec-Qwen2VL-2B


[2026-01-01 10:34:23,190] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/models/TIGER-Lab/VLM2Vec-Qwen2VL-2B/revision/main HTTP/1.1" 200 6640
[2026-01-01 10:34:23,205] DEBUG [urllib3.connectionpool:1049] Starting new HTTPS connection (1): huggingface.co:443
Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s][2026-01-01 10:34:26,742] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-2B/resolve/7717deedf0631e6f520b7c83c8f82dcbc2c4c21e/preprocessor_config.json HTTP/1.1" 307 0
[2026-01-01 10:34:26,951] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-2B/7717deedf0631e6f520b7c83c8f82dcbc2c4c21e/preprocessor_config.json HTTP/1.1" 200 0
[2026-01-01 10:34:26,955] DEBUG [filelock:331] Attempting to acquire lock 134566049429840 on /root/.cache/huggingface/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-2B/2ef5e95d7d22f4b301fbf6bd6de694f5918a33fd.lock
[202

ImageProcessor type: <class 'transformers.models.qwen2_vl.image_processing_qwen2_vl.Qwen2VLImageProcessor'>
teacher processor loaded here.


[2026-01-01 10:35:21,217] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-2B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-01 10:35:21,428] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-2B/7717deedf0631e6f520b7c83c8f82dcbc2c4c21e/config.json HTTP/1.1" 200 0
[2026-01-01 10:35:21,436] DEBUG [filelock:331] Attempting to acquire lock 134566049686608 on /root/.cache/huggingface/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-2B/85b97e98aba8ab3f53cefd874cf7b6c574e3221c.lock
[2026-01-01 10:35:21,440] DEBUG [filelock:334] Lock 134566049686608 acquired on /root/.cache/huggingface/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-2B/85b97e98aba8ab3f53cefd874cf7b6c574e3221c.lock
[2026-01-01 10:35:22,357] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-2B/7717deedf0631e6f520b7c83c8f82dcbc2c4c21e/config.json HTT

Detected model type: qwen2_vl
Determined model backbone: qwen2_vl


[2026-01-01 10:35:22,757] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /Qwen/Qwen2-VL-2B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-01 10:35:22,966] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/Qwen/Qwen2-VL-2B/d3a53f2484fce9d62fff115a5ddfc833f873bfde/config.json HTTP/1.1" 200 0
[2026-01-01 10:35:22,971] DEBUG [filelock:331] Attempting to acquire lock 134566059150416 on /root/.cache/huggingface/hub/.locks/models--Qwen--Qwen2-VL-2B/f74c2d569aa6de68f4c3c70d427ca2eeb9ab802c.lock
[2026-01-01 10:35:22,973] DEBUG [filelock:334] Lock 134566059150416 acquired on /root/.cache/huggingface/hub/.locks/models--Qwen--Qwen2-VL-2B/f74c2d569aa6de68f4c3c70d427ca2eeb9ab802c.lock
[2026-01-01 10:35:23,250] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/resolve-cache/models/Qwen/Qwen2-VL-2B/d3a53f2484fce9d62fff115a5ddfc833f873bfde/config.json HTTP/1.1" 200 None
[2026-01-01 10:35:23,256] DEBUG [filelock:36

KeyboardInterrupt: 

In [ ]:
model.model_backbone

'llava_qwen2'

In [6]:
model.config

LlavaConfig {
  "architectures": [
    "LlavaQwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "llava_qwen.LlavaConfig",
    "AutoModelForCausalLM": "llava_qwen.LlavaQwen2ForCausalLM"
  },
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "freeze_mm_mlp_adapter": false,
  "hidden_act": "silu",
  "hidden_size": 896,
  "image_aspect_ratio": "pad",
  "image_grid_pinpoints": null,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention

In [14]:
vision_tower = model.encoder.get_vision_tower()
vision_tower.config['image_cfg']['patch_size']

64

In [ ]:
x = torch.randn(1, 3, 768, 768).to('cuda', dtype=torch.bfloat16)
with torch.no_grad():
    y = model.encoder.get_vision_tower()(x)
y.shape

RuntimeError: Given input size: (3072x12x12). Calculated output size: (3072x0x0). Output size is too small